<a href="https://colab.research.google.com/github/Bryan-Az/Neurobytes/blob/main/notebooks/test_spotify_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
# access colab secrets .env
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

True

# Data Related to the Spotify API

Main Discovery Objective:
To identify data that can provide Song-to-Song similarity information based on musicality.


---

The spotify API is able to retrieve data for select artists, albums and shows and other collections within Spotify's content like podcasts or playlists that may help us build alternative sets of information outside of the API from where we can begin.

In [10]:
import os
SPOTIFY_CLIENT_ID = os.getenv('client_id')
SPOTIFY_CLIENT_SECRET = os.getenv('client_secret')

In [88]:
def get_access_token(client_id, client_secret):
    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    payload = {
        'grant_type': 'client_credentials'
    }
    response = requests.post(url, headers=headers, data=payload, auth=(client_id, client_secret))
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        raise Exception("Failed to retrieve access token")


In [89]:
access_token = get_access_token(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)

## Artist-Specific Data

In [95]:
def get_artist_data(access_token, artist_id):
    '''
    This function retrieves artist data from the Spotify API using the provided access token and artist ID.
    '''
    url = f'https://api.spotify.com/v1/artists/{artist_id}'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        if response.status_code == 404:
            raise Exception("Artist not found")
        elif response.status_code == 401:
            get_access_token(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)
            return get_artist_data(access_token, artist_id)
        else:
          raise Exception("Failed to retrieve artist data")


In [91]:
artist_id = '1vCWHaC5f2uS3yhpwWbIA6'  # This is the artist ID for Avicii
artist_data = get_artist_data(access_token, artist_id)
print(artist_data)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1vCWHaC5f2uS3yhpwWbIA6'}, 'followers': {'href': None, 'total': 22658238}, 'genres': ['dance pop', 'edm', 'pop', 'pop dance'], 'href': 'https://api.spotify.com/v1/artists/1vCWHaC5f2uS3yhpwWbIA6', 'id': '1vCWHaC5f2uS3yhpwWbIA6', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebae07171f989fb39736674113', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174ae07171f989fb39736674113', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178ae07171f989fb39736674113', 'width': 160}], 'name': 'Avicii', 'popularity': 78, 'type': 'artist', 'uri': 'spotify:artist:1vCWHaC5f2uS3yhpwWbIA6'}


In [96]:
artist_data.keys()

dict_keys(['external_urls', 'followers', 'genres', 'href', 'id', 'images', 'name', 'popularity', 'type', 'uri'])

In [127]:
artist_data['genres']

['dance pop', 'edm', 'pop', 'pop dance']

In [122]:
def get_artist_albums(access_token, artist_id):
  '''
  This function retrieves a list of albums for a given artist using the Spotify API.
  returns: list of albums for the artist
  '''
  # part 1 extracts the album information without the tracks
  include_groups = 'album'
  url = f'https://api.spotify.com/v1/artists/{artist_id}/albums/?include_groups={include_groups}'
  headers = {
      'Authorization': f'Bearer {access_token}'
  }
  response = requests.get(url, headers=headers)
  if response.status_code == 200:
        return response.json()
  else:
      if response.status_code == 404:
          raise Exception("Artist not found")
      elif response.status_code == 401:
          get_access_token(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)
          return 'Try again, new token generated'
      else:
        raise Exception("Failed to retrieve artist data")

In [123]:
# setting include_group to album only to return only this artists own albums
artist_albums = get_artist_albums(access_token, artist_id)

The available metadata available for the artist album's returned.

In [78]:
# might be redundant information if the include_group is set
for i in artist_albums:
  print(i['name'] + ' ' + i['album_group'] + ' '+ i['album_type'])

TIM album album
Stories album album
True: Avicii By Avicii album album
The Days / Nights album album
True (Bonus Edition) album album
True album album
Malo (The Cube Guys Remix) single single
Street Dancer (Sgt Slick's Discotizer 2022 Remix) single single
My Feelings For You (Mark Knight Remix) single single
My Feelings For You (Don Diablo Remix) single single
Forever Yours (Avicii Tribute) single single
Fades Away (feat. MishCatt) [Tribute Concert Version] single single
Heaven (David Guetta & MORTEN Remix) single single
Tough Love (Tiësto Remix) single single
SOS (Laidback Luke Tribute Remix) single single
Tough Love (feat. Vargas & Lagola) single single
SOS (feat. Aloe Blacc) single single
Lonely Together (Remixes) single single
Lonely Together (Acoustic) single single
Without You (Remixes) single single
